<a href="https://colab.research.google.com/github/sashapff/bioinformatics-algorithms/blob/master/alignment/global_affine_penalty.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [230]:
vinf = -10**10

In [303]:
def max_up(left, diag, up, i, j, open_gap_penalty, continue_gap_penalty):
  if up[i - 1][j][0] - continue_gap_penalty > diag[i - 1][j][0] - open_gap_penalty - continue_gap_penalty:
    return (up[i - 1][j][0] - continue_gap_penalty, -1, 0, "up")
  else:
    return (diag[i - 1][j][0] - open_gap_penalty - continue_gap_penalty, -1, 0, "diag")

In [304]:
def max_left(left, diag, up, i, j, open_gap_penalty, continue_gap_penalty):
  if left[i][j - 1][0] - continue_gap_penalty > diag[i][j - 1][0] - open_gap_penalty - continue_gap_penalty:
    return (left[i][j - 1][0] - continue_gap_penalty, 0, -1, "left")
  else:
    return (diag[i][j - 1][0] - open_gap_penalty - continue_gap_penalty, 0, -1, "diag")

In [305]:
def max_diag(left, diag, up, seq1, seq2, i, j, weight_match, weight_mismatch, open_gap_penalty, continue_gap_penalty):
  if seq1[i - 1] == seq2[j - 1]:
    match = weight_match
  else:
    match = -weight_mismatch 
  if (i == 0) and (j == 0):
    return (0, "start", "start")
  if (i > 0) and (j > 0) and (diag[i - 1][j - 1][0] + match > up[i][j][0]) and (diag[i - 1][j - 1][0] + match > left[i][j][0]):
    return (diag[i - 1][j - 1][0] + match, -1, -1, "diag")
  elif up[i][j][0] > left[i][j][0]:
    return (up[i][j][0], 0, 0, "up")
  else:
    return (left[i][j][0], 0, 0, "left")

In [306]:
def build_matrixes(seq1, seq2, weight_match, weight_mismatch, open_gap_penalty, continue_gap_penalty):
  n = len(seq1) + 1
  m = len(seq2) + 1
  left, diag, up = [[(vinf, "") for j in range(m)] for i in range(n)], [[(vinf, "") for j in range(m)] for i in range(n)], [[(vinf, "") for j in range(m)] for i in range(n)]
  for i in range(n):
    for j in range(m):
      if i > 0:
        up[i][j] = max_up(left, diag, up, i, j, open_gap_penalty, continue_gap_penalty)
      if j > 0:
        left[i][j] = max_left(left, diag, up, i, j, open_gap_penalty, continue_gap_penalty)
      diag[i][j] = max_diag(left, diag, up, seq1, seq2, i, j, weight_match, weight_mismatch, open_gap_penalty, continue_gap_penalty)
  return (left, diag, up)

In [315]:
def find_path(seq1, seq2, left, diag, up):
  n = len(seq1) + 1
  m = len(seq2) + 1
  i = n - 1
  j = m - 1
  if (left[-1][-1][0] > up[-1][-1][0]) and (left[-1][-1][0] > diag[-1][-1][0]):
    next = "left"
  elif up[-1][-1][0] > diag[-1][-1][0]:
    next = "up"
  else:
    next = "diag"
  ans1, ans2 = "", ""
  while not((i == 0) and (j == 0)):
    if next == "diag":
      di, dj, next = diag[i][j][1:]
    elif next == "up":
      di, dj, next = up[i][j][1:]
    else:
      di, dj, next = left[i][j][1:]

    if di == -1:
      ans1 += seq1[i - 1]
      i -= 1
    elif dj == -1:
      ans1 += "_"
    if dj == -1:
      ans2 += seq2[j - 1]
      j -= 1
    elif di == -1:
      ans2 += "_"
  ans1 = ans1[::-1]
  ans2 = ans2[::-1]
  return (ans1, ans2)

In [308]:
def affine_gap_alignment(seq1, seq2, weight_match, weight_mismatch, open_gap_penalty, continue_gap_penalty):
  left, diag, up = build_matrixes(seq1, seq2, weight_match, weight_mismatch, open_gap_penalty, continue_gap_penalty)
  return find_path(seq1, seq2, left, diag, up)


In [342]:
affine_gap_alignment("GAACATGCAAB", "CATTGTAACB", 3, 1, 0, 1)

('GAACAT_GCAA_B', '___CATTGTAACB')

In [341]:
affine_gap_alignment("GAACATGCAAB", "CATTGTAACB", 3, 1, 4, 2)

('GAACATGCAA_B', 'CAT__TGTAACB')